### Hypothesis: Property-Based Testing

In this notebook, we use property based testing to find issues in our code. [Hypothesis](https://hypothesis.readthedocs.io/en/latest/) is a great library written primarily by [David MacIver](http://www.drmaciver.com/). It introduces methodology similar to [Haskell's Quickcheck](https://hackage.haskell.org/package/QuickCheck), but in Python -- hooray!

The documentation is incredibly useful and the library is used by many other Python libraries you may know and love. It also has the abilities to mock and test `numpy` datatypes. I recommend it *especially* if you are writing any libraries or have shared code across teams (like utils or helpers and so forth).

In [2]:
from hypothesis import given, assume
from hypothesis.strategies import tuples, integers
from hypothesis.extra.fakefactory import fake_factory
import re

### First, we need to write our function, which takes a tuple and finds the range

In [3]:
def calculate_range(tuple_obj):
    return max(tuple_obj) - min(tuple_obj)

## Then, we can define our test and our rules using Hypothesis `strategies` and `given`

In [4]:
@given(tuples(integers(), integers()))
def test_calculate_range(tup):
    result = calculate_range(tup)
    assert isinstance(result, int)
    assert result > 0

In [5]:
test_calculate_range()

Falsifying example: test_calculate_range(tup=(0, 0))


AssertionError: 

## We can fix our test by adding `>=`

In [6]:
@given(tuples(integers(), integers()))
def test_calculate_range(tup):
    result = calculate_range(tup)
    assert isinstance(result, int)
    assert result >= 0

In [7]:
test_calculate_range()

## You can also use Hypothesis alongside `faker` a library which generates mock data based on specified types.

In [8]:
def parse_email(email):
    result = re.match('(?P<username>\w+)@(?P<domain>\w+)', email).groups()
    return result

In [9]:
@given(fake_factory('email'))
def test_parse_email(email):
    result = parse_email(email)
    #print(result)
    assert len(result) == 2
    assert '.' in result[1]

In [10]:
test_parse_email()

Falsifying example: test_parse_email(email='brigita22@gmail.com')


AssertionError: 

In [16]:
def parse_email(email):
    result = re.match('(?P<username>\w+).(?P<domain>[\w\.]+)', email).groups()
    return result

### fails becuase regex contains a doesn't handle dashes, for email there are api's that can help you

In [17]:
test_parse_email()

Falsifying example: test_parse_email(email='myrto.ziros@papandrikopoulou.gr')


AssertionError: 

## Exercise: can you fix the regex for this error?

In [20]:
# %load ../solutions/hypothesis.py
def parse_email(email):
    result = re.match('(?P<username>[\w\-\.]+).(?P<domain>[\w\.\-]+)', email).groups()
    return result


In [21]:
test_parse_email()